First of all we're going to analyse each variable one by one and conduct mind experiment of whether any dependencies between y and x exist. Also we're going to note some potential ideas about feature engineering. But firstly let's check how many missing data do we have.

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.axes import Axes
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import kpss
import scipy.stats
from scipy.stats import spearmanr
from scipy.stats import ks_2samp
import itertools
from itertools import combinations
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import RFECV
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import f_classif
from scipy.stats import mannwhitneyu
import sys
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn.model_selection import cross_validate
from sklearn import metrics   #Additional scklearn functions
from sklearn.model_selection import GridSearchCV   #Perforing grid search

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
train_set=pd.read_csv(r'../input/train.csv')
train=train_set.drop('SalePrice',axis=1).drop('Id',axis=1)

y_train=train_set.SalePrice
test_set=pd.read_csv(r'../input/test.csv').drop('Id',axis=1)
#NAs=train_set.isnull().sum()
#NAs[NAs!=0]

train_test = pd.concat([train_set,test_set],axis=0).reset_index(drop=True)



Nominal data = ['MSSubClass','MSZoning','LotShape','LandContour','LotConfig','LandSlope','Neighborhood','Condition1','Condition2',
               'BldgType','HouseStyle','YearBuilt','YearRemodAdd','RoofStylec','RoofStyle','RoofMatl', 'Exterior1st','Exterior2nd',
               'MasVnrType','Foundation', 'Heating', 'Electrical','GarageYrBlt','MiscFeature','MoSold','YrSold','SaleType', 'SaleCondition']

Ordinal data = ['Street','Alley','Utilities','OverallQual','OverallCond','ExterQual','ExterCond','BsmtQual','BsmtCond','BsmtExposure',
               'BsmtFinType1', 'BsmtFinType2', 'HeatingQC', 'CentralAir','KitchenQual', 'FireplaceQu','GarageType','GarageFinish',
               'GarageQual','GarageCond','PavedDrive','PoolQuality','Fence']

Interval data = ['SalePrice','LotFrontage','LotArea','MasVnrArea','BsmtFinSF1','BsmtFinSF2','BsmtUnfSF','TotalBsmtSF','1stFlrSF', '1stFlrSF',
                'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath','FullBath','HalfBath','Bedroom','Kitchen','TotRmsAbvGrd',
                'Fireplaces', 'GarageCars', 'GarageArea','WoodDeckSF','OpenPorchSF', 'EnclosedPorchSF', '3SsnPorch', 'ScreenPorch', 
                'PoolArea','MiscVal']

Comments:
a) LandSlope may be ordinal due to harder building conditions on slope terrain

b) LandContour may be similar variable to LandSlope both to categorazation as well as thing that they descriebe
c) Condition 1 and 2 might be ordinal if you investigate them more perciselly. Overall they are the same

d) HouseStyle might be ordinal but it is neccasary to understand deeper what split foyer and split level is. I concider even
creating separate variable for them if i place that variable as ordinal

e) Exterior1st and Exterior2nd descriebes same thing. You may connect them somehow

g) ExternalQuality and ExternalCond sounds like they descriebe same thing. How something might be good quality if it has bad condition?

h) Same with Overall

i) Same with basement

g) TotalBsmtSF=BsmtFinSF1 + BsmtFinSF2 + BsmtUnfSF

h) Also, BsmtFinType1 and 2 descriebe same thing (like Ext in e point)

i) GrLivArea= 1stFlrSF + 2stFlrSF 

j) Connect BsmtFullBath with BsmtHalfBath

k) TotRmsAbvGrd not necassary, because we have split on paticular rooms ?

l) GarageType you may consider it as nominal or not

m) If garage exist and garage year built is not added, then assume that garage year built is house year built

n) GarageQual and Garage cond as h and i point

o) May concider merging Porch variables

p) Fence may also be nominal

In [ ]:
fig, axarr = plt.subplots(1,2,figsize=(17,7))
sns.distplot(train_set.SalePrice,ax=axarr[0])
scipy.stats.probplot(train_set.SalePrice, plot=plt)
print('Skewness=',scipy.stats.skew(train_set.SalePrice))

We're going to deal with this skewness later on. Now we will check distriution of NANs between test and train.



In [ ]:
# fig, axarr = plt.subplots(1,2,figsize=(17,7))
# for i, missing in enumerate([train_set,test_set]):
#     missing_1 = missing.isnull().sum()
#     missing_1[missing_1!=0].sort_values(ascending=False).plot.bar(ax=axarr[i])

We're going to fill NAs right now.

In [ ]:
missing_1 = train_test.isnull().sum()
print(missing_1[missing_1!=0].sort_values(ascending=False).rename('Train missing'))

'''missing_1 = test_set.isnull().sum()
print(missing_1[missing_1!=0].sort_values(ascending=False).rename('Test missing'))'''

In [ ]:
train_test =train_test.replace({'MSSubClass':{
        20:'1-STORY 1946 & NEWER ALL STYLES',
        30:'1-STORY 1945 & OLDER',
        40:'1-STORY W/FINISHED ATTIC ALL AGES',
        45:'1-1/2 STORY - UNFINISHED ALL AGES',
        50:'1-1/2 STORY FINISHED ALL AGES',
        60:'2-STORY 1946 & NEWER',
        70:'2-STORY 1945 & OLDER',
        75:'2-1/2 STORY ALL AGES',
        80:'SPLIT OR MULTI-LEVEL',
        85:'SPLIT FOYER',
        90:'DUPLEX - ALL STYLES AND AGES',
       120:'1-STORY PUD (Planned Unit Development) - 1946 & NEWER',
       150:'1-1/2 STORY PUD - ALL AGES',
       160:'2-STORY PUD - 1946 & NEWER',
       180:'PUD - MULTILEVEL - INCL SPLIT LEV/FOYER',
       190:'2 FAMILY CONVERSION - ALL STYLES AND AGES'}})

'''test_set =test_set.replace({'MSSubClass':{
        20:'1-STORY 1946 & NEWER ALL STYLES',
        30:'1-STORY 1945 & OLDER',
        40:'1-STORY W/FINISHED ATTIC ALL AGES',
        45:'1-1/2 STORY - UNFINISHED ALL AGES',
        50:'1-1/2 STORY FINISHED ALL AGES',
        60:'2-STORY 1946 & NEWER',
        70:'2-STORY 1945 & OLDER',
        75:'2-1/2 STORY ALL AGES',
        80:'SPLIT OR MULTI-LEVEL',
        85:'SPLIT FOYER',
        90:'DUPLEX - ALL STYLES AND AGES',
       120:'1-STORY PUD (Planned Unit Development) - 1946 & NEWER',
       150:'1-1/2 STORY PUD - ALL AGES',
       160:'2-STORY PUD - 1946 & NEWER',
       180:'PUD - MULTILEVEL - INCL SPLIT LEV/FOYER',
       190:'2 FAMILY CONVERSION - ALL STYLES AND AGES'}})'''


train_test['MiscFeature'] = train_test['MiscFeature'].fillna('No_misc')
train_test['MasVnrType'] = train_test['MasVnrType'].fillna('None')

train_test.MiscFeature.loc[train_test[(train_test.MiscFeature=='No_misc') & (train_test.MiscVal>0)].index] = 'Othr'
train_test.MiscVal.loc[train_test[(train_test.MiscFeature=='Shed') & (train_test.MiscVal==0)].index] = 500
train_test.MiscFeature[(train_test.MiscFeature=='Othr') & (train_test.MiscVal==0)] = 'No_misc'

train_test['PoolQC'] = train_test['PoolQC'].fillna('No_pool')
train_test = train_test.replace({'PoolQC':{'Ex':4,'Gd':3,'TA':2,'Fa':1,'No_pool':0}})

train_test.PoolQC.loc[[2420,2503,2599]] = 3


In [ ]:
train_test['Alley'] = train_test['Alley'].fillna('Doesnt_exist')
train_test['Fence'] = train_test['Fence'].fillna('Doesnt_exist')
train_test['FireplaceQu'] = train_test['FireplaceQu'].fillna(0)

In [ ]:
garage = ['GarageType', 'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual', 'GarageCond']

train_test['GarageYrBlt'].loc[2576] = 1923
train_test['GarageYrBlt'].loc[2126] = 1910


In [ ]:
# fig, axarr = plt.subplots(1,2,figsize=(14,7))
# sns.scatterplot(x='GarageYrBlt',y='YearBuilt',data=train_test,ax=axarr[0])
# sns.scatterplot(x='GarageYrBlt',y='YearRemodAdd',data=train_test,ax=axarr[1])

# #YearBuilt 	YearRemodAdd

Some garages were built before whole property was built. I know that this situation may happen, but I assume that it doesn't. Also we have one observation with year of garage built higher than 2200.

In [ ]:
train_test['GarageYrBlt'][train_test.YearBuilt>train_test.GarageYrBlt] =\
train_test['YearBuilt'][train_test.YearBuilt>train_test.GarageYrBlt]
train_test.GarageYrBlt.loc[2592] = 2007

In [ ]:
#train_test = train_test.replace({'GarageFinish':{'Fin':3,'RFn':2,'Unf':1,'Doesnt_exist':0}})
train_test['GarageFinish'].loc[[2576,2126]] = 1
train_test['GarageCars'].loc[[2576,2126]] = 1
train_test['GarageArea'].loc[[2576]] = 288
train_test['GarageQual'].loc[[2576,2126]] = 'TA'
train_test['GarageCond'].loc[[2576,2126]] = 'TA'
train_test[garage] = train_test[garage].fillna('No_garage')
train_test = train_test.replace({'GarageType':{'2Types':6,'Attchd':5,'Basment':4,'BuiltIn':3,'CarPort':2,'Detchd':1,'No_garage':0},
                                 'GarageQual':{'Ex':5,'Gd':4,'TA':3,'Fa':2,'Po':1,'No_garage':0},
                                 'GarageCond':{'Ex':5,'Gd':4,'TA':3,'Fa':2,'Po':1,'No_garage':0},
                                 'GarageFinish':{'Fin':3,'RFn':2,'Unf':1,'No_garage':0}})

As GarageCars we've taken the most frequent value for houses older than 1930. For GarageArea we've aken median of GarageArea for garages with GarageCars equal to 1. For GarageQual and GarageCond we're taking 'TA', because this is most often value for garages built before 1930. Also 2610 observations has value of GarageCond equal to GarageQual. Rest of the observation does not have garage, so we can imput 'No_garage'.

In [ ]:
basement = ['BsmtCond', 'BsmtExposure','BsmtQual','BsmtFinType2','BsmtFinType1','BsmtFinSF2','BsmtFinSF1','TotalBsmtSF','BsmtUnfSF']
#2040, 2524, 2185, 2040 <-- imput BsmtCond
train_test = train_test.replace({'BsmtQual':{'Ex':5,'Gd':4,'TA':3,'Fa':2,'Po':1},
                                 'BsmtCond':{'Ex':5,'Gd':4,'TA':3,'Fa':2,'Po':1},
                                 'BsmtExposure':{'Gd':4,'Av':3, 'Mn':2,'No':1,},
                                 'BsmtFinType1':{'GLQ':6,'ALQ':5,'BLQ':4,'Rec':3,'LwQ':2,'Unf':1},
                                 'BsmtFinType2':{'GLQ':6,'ALQ':5,'BLQ':4, 'Rec':3,'LwQ':2, 'Unf':1}})
train_test.BsmtCond.loc[[2040, 2524, 2185, 2040]] = 3
#2348, 1487, 948 <-- imput BsmtExposure
train_test.BsmtExposure.loc[[2348,1487,948]] = 3
#2217, 2218
train_test.BsmtQual.loc[[2217, 2218]] = 3
train_test['BsmtFinSF1'].loc[2284] = 638
train_test['BsmtFinSF2'].loc[2284] = 0
train_test['BsmtFinType2'].loc[332] = 1
train_test[['BsmtFinType2', 'BsmtCond', 'BsmtQual', 'BsmtExposure', 'BsmtFinType1']]=\
train_test[['BsmtFinType2', 'BsmtCond', 'BsmtQual', 'BsmtExposure', 'BsmtFinType1']].fillna(0)
train_test[['BsmtFinType2','BsmtFinType2']]=train_test[['BsmtFinType2','BsmtFinType2']].fillna(0)
train_test[['BsmtFinSF1','BsmtFinSF2','TotalBsmtSF','BsmtUnfSF','BsmtHalfBath','BsmtFullBath']]=\
train_test[['BsmtFinSF1','BsmtFinSF2','TotalBsmtSF','BsmtUnfSF','BsmtHalfBath','BsmtFullBath']].fillna(0)


Beacause of weak correlations of BsmtCond, BsmtExposure we take mode value as impot for observations with NANs. For houses where we miss BsmtQual, but we know that basement appears we imput 3 as Quality (based on high correlation between YearBuilt and BsmtQual. Also houses below 1930 have median quality of 3). Also, there is one basement in which in my opinion it was wrongly declared that it has more than one finish type. One house with basement, but with missing BsmtFinType2 value received 1, because this is the most often appering value for houses where sum of basement finished is greater or equal basement unfinished area. Rest of the houses receive 0 value, because they just don't have basement.

In [ ]:
train_test.MSZoning.loc[[1915,2216,2250]] = 'RM'
train_test.MSZoning.loc[2904] = 'RL'

Here I'm imputing these values, because majority of observations from IDOTRR neighborhood live in RM. Same logic with Mitchel, but there is 90/10 proportion to RL

In [ ]:
train_test = train_test.replace({'KitchenQual':{'Ex':5,'Gd':4,'TA':3,'Fa':2,'Po':1}})

train_test.Utilities = train_test.Utilities.fillna('AllPub')
train_test.Functional = train_test.Functional.fillna('Typ')
train_test.SaleType = train_test.SaleType.fillna('WD')
train_test.KitchenQual.loc[1555] = 3
train_test['Electrical'] = train_test['Electrical'].fillna('SBrkr')
train_test[['Exterior1st','Exterior2nd']] = train_test[['Exterior1st','Exterior2nd']].fillna('VinylSd')

Almost every obserbation have all utilities so I'm going to assume that NAs also have them. For KitchenQual we imput 3, because this is most frequently appearing number in this neighborhood

In [ ]:
a = train_test[['OverallQual','MasVnrArea']][train_test.MasVnrArea>0]
a[['OverallQual','MasVnrArea']].groupby('OverallQual').median()

train_test[['OverallQual','MasVnrArea']].groupby('OverallQual').median()
train_test['MasVnrArea'][(train_test.MasVnrArea.isnull()) & (train_test.OverallQual==6)] = 203.0
train_test['MasVnrArea'][(train_test.MasVnrArea.isnull()) & (train_test.OverallQual==7)] = 174
train_test['MasVnrArea'][(train_test.MasVnrArea.isnull()) & (train_test.OverallQual==8)] = 257
train_test['MasVnrArea'][(train_test.MasVnrArea.isnull()) & (train_test.OverallQual==10)] = 692

In masony veneer area we're imputing median of area in each OverallQual category.

In [ ]:
# train_test.dropna().corr(method='spearman').LotFrontage.sort_values(ascending=False).plot.bar()

In [ ]:
# train_test.LotFrontage = train_test.LotFrontage.fillna(train_test.LotFrontage.median())

In [ ]:
a=train_test[(train_test.LotShape=='Reg') | (train_test.LotShape=='IR1')]
a=a[['LotFrontage','LotArea','LotShape']]
b=a.LotFrontage**2
a=a.rename(columns={'LotFrontage':'LotFrontage^1'})
a=pd.concat([a,b],axis=1)
a=a.rename(columns={'LotFrontage':'LotFrontage^2'})
c=a['LotFrontage^2']/a.LotArea
a=pd.concat([a,c],axis=1)
a=a.rename(columns={0:'Percentage_share_^2'})
a=a.dropna()

train_test['LotFrontage'][(train_test.LotFrontage.isnull()) & (train_test.LotShape=='Reg')]=\
(train_test.LotArea[train_test.LotFrontage.isnull()]**(1/2))*(np.median(a['Percentage_share_^2'][a.LotShape=='Reg'])**(1/2))

train_test['LotFrontage'][(train_test.LotFrontage.isnull()) & (train_test.LotShape=='IR1')]=\
(train_test.LotArea[train_test.LotFrontage.isnull()]**(1/2))*(np.median(a['Percentage_share_^2'][a.LotShape=='IR1'])**(1/2))

In [ ]:
# # plt, axarr = plt.subplots(figsize=(14,7))
# # train_test.dropna().corr()['LotFrontage'].sort_values(ascending = False).plot.bar()
# a = train_test[['LotFrontage','LotArea']][(train_test.LotFrontage <300) & (train_test.LotArea <100000)]

# a.dropna().plot.scatter(x='LotArea', y='LotFrontage')

In [ ]:
train_test = train_test.replace({'Street':{'Pave':1,'Grvl':0},
                             'Alley':{'Doesnt_exist':0,'Grvl':1,'Pave':2},
                             'Fence':{'GdPrv':4,'MnPrv':3,'GdWo':2,'MnWw':1,'Doesnt_exist':0},
                             'Utilities':{'AllPub':4,'NoSewr':3,'NoSeWa':2,'ELO':1},
                             'ExterQual':{'Ex':5,'Gd':4,'TA':3,'Fa':2,'Po':1},
                             'ExterCond':{'Ex':5,'Gd':4,'TA':3,'Fa':2,'Po':1},
                             'HeatingQC':{'Ex':5,'Gd':4,'TA':3,'Fa':2,'Po':1},
                             'CentralAir':{'Y':1,'N':0},
                             'Functional':{'Typ':8,'Min2':7, 'Min1':6,'Mod':5,'Maj1':4,'Maj2':3,'Sev':2,'Sal':1},
                             'PavedDrive':{'Y':3,'N':2,'P':1},
                             'FireplaceQu':{'Ex':5,'Gd':4,'TA':3,'Fa':2,'Po':1}})

Here we just changing ordinal scale so it is represented by numbers

In [ ]:
train_test['LotFrontage'][(train_test.LotFrontage.isnull()) & (train_test.LotShape=='IR2') ] = \
np.median(train_test.LotFrontage[train_test.LotShape=='IR2'].dropna())
train_test['LotFrontage'][(train_test.LotFrontage.isnull()) & (train_test.LotShape=='IR3') ] = \
np.median(train_test.LotFrontage[train_test.LotShape=='IR3'].dropna())

Here we just adding median from IR2 and IR3 as LotFrontage for frontages with missing values

Before we will conver months into nominal variables, we're going to check whether we may assume that time series is stationary.

In [ ]:
train_test.MoSold[train_test.YrSold==2010].value_counts()

In [ ]:
train_set_copy=train_set[['SalePrice','YrSold','MoSold']]
train_set_copy1=train_set_copy[train_set_copy.YrSold==2006]
train_set_copy2=train_set_copy[train_set_copy.YrSold==2007].\
replace({'MoSold':{1:13,2:14,3:15,4:16,5:17,6:18,7:19,8:20,9:21,10:22,11:23,12:24}})
train_set_copy3=train_set_copy[train_set_copy.YrSold==2008].\
replace({'MoSold':{1:25,2:26,3:27,4:28,5:29,6:30,7:31,8:32,9:33,10:34,11:35,12:36}})
train_set_copy4=train_set_copy[train_set_copy.YrSold==2009].\
replace({'MoSold':{1:37,2:38,3:39,4:40,5:41,6:42,7:43,8:44,9:45,10:46,11:47,12:48}})
train_set_copy5=train_set_copy[train_set_copy.YrSold==2010].\
replace({'MoSold':{1:49,2:50,3:51,4:52,5:53,6:54,7:55,8:56,9:57,10:58,11:59,12:60}})

train_set_copy1=train_set_copy1[['SalePrice','MoSold']].groupby('MoSold').mean()
train_set_copy2=train_set_copy2[['SalePrice','MoSold']].groupby('MoSold').mean()
train_set_copy3=train_set_copy3[['SalePrice','MoSold']].groupby('MoSold').mean()
train_set_copy4=train_set_copy4[['SalePrice','MoSold']].groupby('MoSold').mean()
train_set_copy5=train_set_copy5[['SalePrice','MoSold']].groupby('MoSold').mean()

Time_series=train_set_copy1.append([train_set_copy2,train_set_copy3,train_set_copy4,train_set_copy5]).reset_index()
Time_series=Time_series.rename(columns={'MoSold':'t'})
Time_series=Time_series.drop(54,axis=0)

import matplotlib.pyplot

fig,axarr=matplotlib.pyplot.subplots(1,1,figsize=(12,5))
time_series=sns.lineplot(x='t',y='SalePrice',data=Time_series).set(xlabel='Time',ylabel='Sale Price')
matplotlib.pyplot.xticks(range(0,55))
time_series=sns.set_style('whitegrid')


It seems that variance is varying in time. In order to approve intuition we're going to perform two tests: KPSS and ADF

In [ ]:
train_set = train_test.loc[:1459]
train_set = train_set.drop('Id',axis=1)
correlation = train_set.corr(method='spearman').SalePrice.sort_values()
correlation = pd.DataFrame(correlation)
fig, axarr = matplotlib.pyplot.subplots(1,1,figsize=(70,180))
barplot = sns.barplot(data=correlation, y=correlation.index, x='SalePrice', orient='h')
barplot.tick_params(labelsize=60)


In [ ]:
kpss(train_set.SalePrice)

#Augmented Dickey-Fuller unit root test

adfuller(train_set.SalePrice)

#Kwiatkowski–Phillips–Schmidt–Shin

Both tests indicates that it is nothing to worry in terms of non-stationarity

In [ ]:
train_test = train_test.replace({'MoSold':{1:'Jan',2:'Feb',3:'Mar',4:'Apr',5:'May',6:\
                                           'Jun',7:'Jul',8:'Aug',9:'Sep',10:'Oct',11:'Nov',12:'Dec'}})

Now, we're going to look for outliers in the most correlated variables

In [ ]:
fig, axarr=matplotlib.pyplot.subplots(6,3,figsize=(20,30))
sns.scatterplot(x='OverallQual',y='SalePrice',data=train_set,ax=axarr[0][0])
sns.scatterplot(x='GrLivArea',y='SalePrice',data=train_set,ax=axarr[0][1])
sns.scatterplot(x='GarageCars',y='SalePrice',data=train_set,ax=axarr[0][2])
sns.scatterplot(x='ExterQual',y='SalePrice',data=train_set,ax=axarr[1][0])
sns.scatterplot(x='BsmtQual',y='SalePrice',data=train_set,ax=axarr[1][1])
sns.scatterplot(x='KitchenQual',y='SalePrice',data=train_set,ax=axarr[1][2])
sns.scatterplot(x='YearBuilt',y='SalePrice',data=train_set,ax=axarr[2][0])
sns.scatterplot(x='GarageArea',y='SalePrice',data=train_set,ax=axarr[2][1])
sns.scatterplot(x='FullBath',y='SalePrice',data=train_set,ax=axarr[2][2])
sns.scatterplot(x='GarageFinish',y='SalePrice',data=train_set,ax=axarr[3][0])
sns.scatterplot(x='TotalBsmtSF',y='SalePrice',data=train_set,ax=axarr[3][1])
sns.scatterplot(x='1stFlrSF',y='SalePrice',data=train_set,ax=axarr[3][2])
sns.scatterplot(x='YearRemodAdd',y='SalePrice',data=train_set,ax=axarr[4][0])
sns.scatterplot(x='FireplaceQu',y='SalePrice',data=train_set,ax=axarr[4][1])
sns.scatterplot(x='TotRmsAbvGrd',y='SalePrice',data=train_set,ax=axarr[4][2])
sns.scatterplot(x='Fireplaces',y='SalePrice',data=train_set,ax=axarr[5][0])
sns.scatterplot(x='GarageType',y='SalePrice',data=train_set,ax=axarr[5][1])
sns.scatterplot(x='HeatingQC',y='SalePrice',data=train_set,ax=axarr[5][2])

In [ ]:
train_set[(train_set.OverallQual==10) & (train_set.SalePrice<200000)]
train_test = train_test.drop([523,1298],axis=0)
# train_set = train_set.drop([523,1298],axis=0)

I assume that someone forgot about one zero at the end of sale price, but nevertheless, we're not going to implement this assumption - just delete it.

In [ ]:
train_set[(train_set.YearBuilt<1900) & (train_set.SalePrice>400000)]

If it comes to this observation, mainly overall quality, and above grade living area justifies high price. Im going to let it be for now.

In [ ]:
train_set[(train_set.GarageArea>1200) & (train_set.SalePrice<300000)]

Observation 1061 has garage bigger than entire above grade living area, 1190 almost same. Also 581 seems to be to small for this garage, and there isn't anything else that justifies high price

In [ ]:
train_test = train_test.drop([581,1061,1190],axis=0)
# train_set = train_set.drop([581,1061,1190],axis=0)

In [ ]:
train_set[(train_set['YearRemodAdd']<1970) & (train_set.SalePrice>300000)]

In [ ]:
train_test = train_test.drop(313,axis=0)
# train_set = train_set.drop(313,axis=0)

Observation appears in upper limits of most the most correlated variables, so I don't think that there is anything that justifies such high price.

In [ ]:
train_set[(train_set['TotRmsAbvGrd']==14)]

This observation also has big above grade living area. On the other huge drawback may be lack of garage. Anyway, in my opinion its better to delete this observation.

In [ ]:
train_test = train_test.drop(635,axis=0)
# train_set = train_set.drop(635,axis=0)

In [ ]:
# fig, axarr=matplotlib.pyplot.subplots(5,1,figsize=(30,50))
# sns.boxplot(x='OverallQual',y='SalePrice',data=train_set,ax=axarr[0])
# sns.boxplot(x='GarageCars',y='SalePrice',data=train_set,ax=axarr[1])
# sns.boxplot(x='ExterQual',y='SalePrice',data=train_set,ax=axarr[2])
# sns.boxplot(x='BsmtQual',y='SalePrice',data=train_set,ax=axarr[3])
# sns.boxplot(x='KitchenQual',y='SalePrice',data=train_set,ax=axarr[4])


In [ ]:
# Nominal_data = ['MSSubClass','MSZoning','LotShape','LandContour','LotConfig','LandSlope','Neighborhood',\
#                 'Condition1','Condition2', 'BldgType','HouseStyle','YearRemodAdd',\
#                 'RoofStyle','RoofMatl', 'Exterior1st','Exterior2nd', 'MasVnrType','Foundation', 'Heating',\
#                 'Electrical','MiscFeature','SaleType', 'SaleCondition']

# fig, axarr = matplotlib.pyplot.subplots(23,1,figsize=(30,300))
# obj = train_set[Nominal_data]
# obj = pd.concat([obj,train_set.SalePrice],axis=1)
# for i, objects in enumerate(obj.drop('SalePrice',axis=1).columns,start=0):
#     order = obj[['SalePrice',objects]].groupby(objects).median().sort_values(by='SalePrice').index
#     sns.boxplot(y='SalePrice',x=objects,data=obj,ax=axarr[i],order=order)

Based on the above charts i mainly had some thoughts about observations with SalePrice above 700000, but major quantitative variables justify high price. Also, I was wondering about observation from Old Town neighborhood with price over 400000, but other most important quantitative variables justify high price. Overall I was very understanding with outliers.

Obviouslly if model will not be satisfying we may experiment with deleting other outliers.

Now in order to simplify dataset we're going to merge categories with similar distributions. In order to do this we will perform Kolmogorov-Smirnoff two sample test. We will set minimal sample size of the feature to 50 in order to minimize risk of not rejecting null hypotesis based on small sample size.

If certain feature will stay with only one unique value, then it gets deleted.



In [ ]:
categories = ['MSSubClass','MSZoning','LotShape','LandContour','LotConfig','LandSlope','Condition1','Condition2',\
               'BldgType','HouseStyle','RoofStyle','RoofMatl', 'Exterior1st','Exterior2nd',\
               'MasVnrType','Foundation', 'Heating', 'Electrical','MiscFeature','SaleType', 'SaleCondition',\
                'CentralAir','PoolQC']

train_set = train_test[:1460]

i=1
while i!=0:
    i=0
    for category in categories:
        for combination in list(combinations(train_set[category].value_counts().index,2)):
            if len(train_set.SalePrice[train_set[category]==combination[0]])<20 or len(train_set.SalePrice[train_set[category]==combination[1]])<20:
                continue
            else:
                ks_result=mannwhitneyu(train_set.SalePrice[train_set[category]==combination[0]],
                                train_set.SalePrice[train_set[category]==combination[1]])[1]
                
                if ks_result>0.05:
                    train_set[category]=train_set[category].replace(combination[0],'{}+{}'.format(combination[0],combination[1]))
                    train_set[category]=train_set[category].replace(combination[1],'{}+{}'.format(combination[0],combination[1]))
                    i=i+1
                        
                    train_test[category]=train_test[category].replace(combination[0],'{}+{}'.format(combination[0],combination[1]))
                    train_test[category]=train_test[category].replace(combination[1],'{}+{}'.format(combination[0],combination[1]))
        
for feature in categories:
    if len(train_test[feature].value_counts())==1:
        train_test=train_test.drop(feature,axis=1)
        train_set=train_set.drop(feature,axis=1)

In [ ]:
# train_test = train_test.replace({'BsmtFinType1':{'5.0+2.0':2,'4.0+3.0':2,6.0:3},
#                                  'OverallCond':{'6+7+8':6,9:7},
#                                  'BsmtFinType2':{'1.0+2.0':1,'3.0+4.0':2,5.0:3,6.0:4},
#                                  'Functional':{'7+6':6,8:7},
#                                  'PavedDrive':{'2+1':0,3:1},
#                                  'BsmtExposure':{'3.0+2.0':2,4:3},
#                                  'BsmtQual':{'0.0+2.0':0,3:1,4:2},
#                                  'BsmtCond':{'2.0+0.0':0,3.0:1,4:2},
#                                  'GarageCond':{'0+2':0,3:1,4:2},
#                                  'Fence':{'0+4':0,'3+2':1}})

# train_test = train_test.replace({'BsmtQual':{5:3},
#                                 'BsmtCond':{1:0},
#                                 'GarageCond':{5:3}})

# train_set = train_set.replace({'BsmtFinType1':{'5.0+2.0':2,'4.0+3.0':2,6.0:3},
#                                  'OverallCond':{'6+7+8':6,9:7},
#                                  'BsmtFinType2':{'1.0+2.0':1,'3.0+4.0':2,5.0:3,6.0:4},
#                                  'Functional':{'7+6':6,8:7},
#                                  'PavedDrive':{'2+1':0,3:1},
#                                  'BsmtExposure':{'3.0+2.0':2,4:3},
#                                  'BsmtQual':{'0.0+2.0':0,3:1,4:2},
#                                  'BsmtCond':{'2.0+0.0':0,3.0:1,4:2},
#                                  'GarageCond':{'0+2':0,3:1,4:2},
#                                  'Fence':{'0+4':0,'3+2':1}})

# train_set = train_set.replace({'BsmtQual':{5:3},
#                                 'BsmtCond':{1:0},
#                                 'GarageCond':{5:3}})

In [ ]:
# correlation_update = train_set[['BsmtFinType1','OverallCond','BsmtFinType2','Functional','PavedDrive',\
#                                 'BsmtExposure','BsmtQual','BsmtCond','GarageCond','Fence','SalePrice']].corr()['SalePrice']

# correlation = correlation.loc[['BsmtFinType1','OverallCond','BsmtFinType2','Functional','PavedDrive',\
#                                 'BsmtExposure','BsmtQual','BsmtCond','GarageCond','Fence','SalePrice']]
# (np.absolute(correlation_update) > np.absolute(correlation))

After transformation by using KS two sample test every absolute value of correlation between feature and SalePrice has increased. Because of that i think, that this change was reasonable.

Now, we're going to make some feature engineering along with removing skewnessess and stuff.

In [ ]:
basement = ['BsmtCond', 'BsmtExposure','BsmtQual','BsmtFinType2','BsmtFinType1','BsmtFinSF2','BsmtFinSF1','TotalBsmtSF','BsmtUnfSF']
fig, axarr = plt.subplots(9,1,figsize=(20,70))
sns.distplot(train_test[basement[0]],kde=False,bins=6,ax = axarr[0])
sns.distplot(train_test[basement[1]],kde=False,bins=6,ax = axarr[1])
sns.distplot(train_test[basement[2]],kde=False,bins=4,ax = axarr[2])
sns.distplot(train_test[basement[3]],kde=False,bins=4,ax = axarr[3])
sns.distplot(train_test[basement[4]],kde=False,bins=4,ax = axarr[4])
sns.distplot(train_test[basement[5]],kde=False,bins=4,ax = axarr[5])
sns.distplot(train_test[basement[6]],kde=False,bins=4,ax = axarr[6])
sns.distplot(train_test[basement[7]],kde=False,bins=10,ax = axarr[7],)
sns.distplot(train_test[basement[8]],kde=False,bins=6,ax = axarr[8])

#Basement binning

train_test['TotalBsmtSF'].loc[2549] = 3200
train_test['BsmtCondBin*'] = train_test.BsmtCond.map(lambda x: 2 if x>2 else 1)
train_test['BsmtExposureBin*'] = train_test.BsmtExposure.map(lambda x: 2 if x>1 else 1)
train_test['BsmtQualBin*'] = 1
train_test['BsmtQualBin*'] = train_test.BsmtQual.map(lambda x: 3 if x>2 else 1)
train_test['TotalBsmtSFBin*'] = pd.cut(train_test.TotalBsmtSF,10, labels=range(1,11)).astype(int)
train_test['BsmtUnfSFBin*'] = pd.cut(train_test.BsmtUnfSF,10, labels=range(1,11)).astype(int)
train_test['TotalBsmtSFBinRoot*'] = train_test['TotalBsmtSFBin*']**(1/2)
train_test['BsmtUnfSFBinRoot*'] = train_test['BsmtUnfSFBin*']**(1/2)

In [ ]:
fig, axarr = plt.subplots(7,1,figsize=(20,50))
garage = ['GarageType','GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual', 'GarageCond']
sns.distplot(train_test[garage[0]],kde=False,bins=3,ax = axarr[0])
sns.distplot(train_test[garage[1]].replace('No_garage',1800),kde=False,bins=2,ax = axarr[1])
sns.distplot(train_test[garage[2]],kde=False,bins=4,ax = axarr[2])
sns.distplot(train_test[garage[3]],kde=False,bins=2,ax = axarr[3])
sns.distplot(train_test[garage[4]],kde=False,bins=6,ax = axarr[4])
sns.distplot(train_test[garage[5]],kde=False,bins=4,ax = axarr[5])
sns.distplot(train_test[garage[6]],kde=False,bins=4,ax = axarr[6])


train_test['GarageTypeBin*'] = pd.cut(train_test.GarageType,3,labels=[1,2,3]).astype(int)
train_test['GarageYrBltBin*'] = pd.cut(train_test[garage[1]].replace('No_garage',1800),2,labels=[1,2]).astype(int)
train_test['GarageFinishBin*'] = pd.cut(train_test.GarageFinish,4,labels=[1,2,3,4]).astype(int)
train_test['GarageCarsBin*'] = pd.cut(train_test.GarageCars,2,labels=[1,2]).astype(int)
train_test['GarageAreaBin*'] = pd.cut(train_test.GarageArea,6,labels=[1,2,3,4,5,6]).astype(int)
train_test['GarageQualBin*'] = train_test.GarageQual.map(lambda x: 2 if x>2 else 1)
train_test['GarageCondBin*'] = train_test.GarageCond.map(lambda x: 2 if x>2 else 1)

train_test['GarageAreaBinRoot*'] = train_test['GarageAreaBin*']**(1/2)

In [ ]:
# train_test = train_test.drop(['BsmtFullBath','BsmtHalfBath'],axis=1)
# train_test = pd.concat([train_test,BsmtBath],axis=1).rename(columns={0:'BsmtBath'})

train_test['GarageQuality*'] = train_test.GarageQual + train_test.GarageCond
train_test['HasPool*'] = 0
train_test['HasPool*'][train_test.PoolQC>0] = 1

train_test['HasGarage*'] = train_test.GarageFinish.map(lambda x: 1 if x>0 else 0)
train_test['HasFireplace*'] = train_test.Fireplaces.map(lambda x: 1 if x>0 else 0)
train_test['Fireplaces*FireplaceQu*'] = train_test.Fireplaces * train_test.FireplaceQu

train_test['HasBasement*'] = train_test.TotalBsmtSF.map(lambda x: 1 if x>0 else 0)


closed = train_test['EnclosedPorch'] + train_test['3SsnPorch'] + train_test['ScreenPorch'] 
opened = train_test['WoodDeckSF'] + train_test['OpenPorchSF']
train_test['PorchArea*'] = closed + opened
train_test['HasPorch*'] = 0
train_test['HasPorch*'] = train_test['PorchArea*'].map(lambda x: 1 if x>0 else 0)
train_test['PorchClosed*'] = closed.map(lambda x: 1 if x > 0 else 0)

train_test['BsmtFinType*'] = train_test.BsmtFinType2 + train_test.BsmtFinType1
train_test['BsmtUnfinPercent*'] = train_test.BsmtUnfSF / train_test.TotalBsmtSF
train_test['BsmtUnfinPercent*'][train_test['BsmtUnfinPercent*'].isnull()] = 0
train_test['TotalHouseSF*'] = train_test.TotalBsmtSF + train_test.GrLivArea + train_test.TotalBsmtSF

IsFresh = train_test[(train_test.YearRemodAdd == train_test.YrSold)].index
train_test['IsFresh*'] = 0
train_test['IsFresh*'].loc[IsFresh] = 1

train_test['AmtOfStories*'] = 0
train_test['AmtOfStories*'][train_test.HouseStyle=='1Story'] = 1
train_test['AmtOfStories*'][(train_test.HouseStyle=='1.5Fin') | (train_test.HouseStyle=='1.5Unf')] = 1.5
train_test['AmtOfStories*'][(train_test.HouseStyle=='2.5Fin') | (train_test.HouseStyle=='2.5Unf')] = 2.5
train_test['AmtOfStories*'][(train_test.HouseStyle=='2Story')] = 2

train_test['AllStoriesFinished*'] = 1
train_test['AllStoriesFinished*'][(train_test['HouseStyle']=='1.5Unf') | (train_test['HouseStyle']=='2.5Unf')] = 0


train_test['OverallQuality*'] = train_test.OverallQual * train_test.OverallCond
train_test['BasementQuality*'] = train_test.BsmtCond * train_test.BsmtExposure
train_test['BsmtFinishType*'] = train_test.BsmtFinType1 * train_test.BsmtFinType2
train_test = train_test.drop(['BsmtFinType1', 'BsmtFinType2'], axis=1)
train_test['Bathrooms*'] = train_test.BsmtFullBath + train_test.BsmtHalfBath/2 + train_test.FullBath + train_test.HalfBath/2
train_test['ExteriorQuality*'] = train_test.ExterQual * train_test.ExterCond


train_test.GarageYrBlt = train_test.GarageYrBlt.replace('No_garage',0)
train_test['NewGarage*'] = 0
train_test['NewGarage*'][train_test.GarageYrBlt>2000]=1
# train_test = train_test.drop('GarageYrBlt',axis=1)
# train_test.GarageYrBlt[train_test.GarageYrBlt==0] = train_test.YearBuilt

train_test['HasGarage*'] = train_test.GarageArea.map(lambda x: 1 if x > 0 else 0)

train_test['CanHold2Cars'] = train_test.GarageCars.map(lambda x: 1 if x>=2 else 0)
train_test['BsmtFinSF*'] = train_test.BsmtFinSF1 + train_test.BsmtFinSF2


# train_test = train_test.drop(['BsmtFullBath','BsmtHalfBath','FullBath','HalfBath'],axis=1)
# train_test = train_test.drop(['YrSold','MoSold'],axis=1)
train_test['YearBltandYearRemodAdd'] = train_test.YearBuilt + train_test.YearRemodAdd
train_test = train_test.drop('Utilities',axis=1)


a = train_test[:1459][['Neighborhood','SalePrice']].groupby('Neighborhood').mean().sort_values(by='SalePrice')
train_test['RichNeighborhood'] = 1
ones = a[0:3].index
twos = a[3:12].index
threes = a[12:19].index
fours = a[19:22].index
fives = a[22:].index
i=0
for variable in [ones, twos, threes, fours, fives]:
    i += 1
    for neighborhood in variable:
        train_test['RichNeighborhood'][train_test.Neighborhood==neighborhood] = i

In [ ]:
# # Interval data = [,'LotFrontage','LotArea','MasVnrArea','BsmtFinSF1','BsmtFinSF2','BsmtUnfSF',\
# #                  'TotalBsmtSF','1stFlrSF', '1stFlrSF', 'LowQualFinSF', 'GrLivArea', \
# #                  ,'GarageArea','WoodDeckSF','OpenPorchSF', 'EnclosedPorchSF', '3SsnPorch',\
# #                  'ScreenPorch', 'PoolArea','MiscVal', 'YearBlt', 'YearRemodAdd', 'YearBltandYearRemodAdd','TotalHouseSF*']

# for i in train_test.select_dtypes(exclude=object).columns:
#     train_test['{}SQUA'.format(i)] = train_test[i]**2
#     train_test['{}CUBE'.format(i)] = train_test[i]**3
    
# train_test = train_test.drop('SalePriceSQUA',axis=1)
# train_test = train_test.drop('SalePriceCUBE',axis=1)

In [ ]:
train_test['LnBsmtQual*'] = np.log(train_test.BsmtQual+1)
train_test['LnExterQual*'] = np.log(train_test.ExterQual)
train_test['LnKitchenQual*'] = np.log(train_test.KitchenQual)
train_test['LnOverallQual*'] = np.log(train_test.OverallQual)
train_test['LnBathrooms*'] = np.log(train_test['Bathrooms*'])
train_test['LnGarageCars*'] = np.log(train_test['GarageCars']+1)
train_test['LnGarageArea*'] = np.log(train_test['GarageArea']+1)
# train_test['StandCbrtGarageArea'] = np.cbrt(((train_test.GarageArea+1)-np.mean(train_test.GarageArea))/np.std(train_test.GarageArea))
train_test['LnTotRmsAbvGrd*'] = np.log(train_test['TotRmsAbvGrd'])
train_test['LnOpenPorchSF*'] = np.log(train_test['OpenPorchSF']+1)
train_test['LnLotArea*'] = np.log(train_test['LotArea'])
train_test['LnPorchArea*'] = np.log(train_test['PorchArea*']+1)
train_test['LnLotFrontage*'] = np.log(train_test['LotFrontage'])
train_test['LnFireplaceQu*'] = np.log(train_test.FireplaceQu + 1)
train_test['LnFireplaces*'] = np.log(train_test.Fireplaces + 1)
train_test['LnGrLivArea*'] = pd.Series(np.log(train_test.GrLivArea))
train_test['Ln1stFlrSF*'] = pd.Series(np.log(train_test['1stFlrSF']))


We connect two columns - BsmtFullBath and BsmtHalfBath in order to provide more informations in one column.

Creates new variable which indicates whether given observation has pool or not. Als wanted to underline here, that correlations between OverallQual and Cond, BsmtQual and Cond, GarageQual and Cond have week correlation, so there is no point in thinking that they descriebe same thing.

In [ ]:
# train_test[train_test['EnclosedPorch']>0] #459
# len(train_test[train_test['3SsnPorch']>0]) #37
# len(train_test[train_test['ScreenPorch']>0]) #256  sum(752)



In [ ]:
# order = train_test[:1459][['Neighborhood','SalePrice']].groupby('Neighborhood').median().sort_values(by='SalePrice').index

# fig, axarr = plt.subplots(figsize=(17,10))
# sns.barplot(x='Neighborhood',y='SalePrice',data=train_test[:1459], order=order)

In [ ]:
# a = train_test[:1459][['Neighborhood','SalePrice']].groupby('Neighborhood').mean().sort_values(by='SalePrice')

# fig, axarr = plt.subplots(figsize=(15,7))
# sns.barplot(x=a.index,y='SalePrice',data=a)

In [ ]:
# a = train_test[:1459][['Neighborhood','SalePrice']].groupby('Neighborhood').mean().sort_values(by='SalePrice')
# train_test['RichNeighborhood'] = 1
# ones = a[0:3].index
# twos = a[3:9].index
# threes = a[9:12].index
# fours = a[12:15].index
# fives = a[15:].index
# i=0
# for variable in [ones, twos, threes, fours, fives]:
#     i += 1
#     for neighborhood in variable:
#         train_test['RichNeighborhood'][train_test.Neighborhood==neighborhood] = i

In [ ]:
# Continous = ['GrLivArea','YearBuilt','GarageArea','TotalBsmtSF','1stFlrSF','PorchArea'
# 'TotalHouseSF']
# fig, axarr = plt.subplots(1,2,figsize=(17,7))

# x = train_test['YearBuilt']

# sns.distplot(x,ax=axarr[0])
# scipy.stats.probplot(x, plot=plt)

In [ ]:
fig, axarr = plt.subplots(1,1,figsize=(14,7),sharex=True)

a = train_set[['SaleType','SalePrice']].groupby('SaleType').median()
train_test.SaleType.value_counts().plot.bar()
print(train_test.SaleType.value_counts())


In [ ]:
train_test = train_test.replace({'BldgType':{'Duplex':'Other', 'Twnhs':'Other', '2fmCon':'Other'},
                                'Electrical':{'FuseF':'Other','FuseP':'Other','Mix':'Other'},
                                 'Foundation':{'Slab':'Other','Stone':'Other','Wood':'Other'},
                                 'Heating':{'Grav':'Other','Wall':'Other','OthW':'Other','Floor':'Other'},
                                 'HouseStyle':{'2.5Unf':'Other','1.5Unf':'Other','2.5Fin':'Other'},
                                 'MSSubClass':{'1-1/2 STORY PUD - ALL AGES':'Other','1-STORY W/FINISHED ATTIC ALL AGES':'Other'\
                                 ,'PUD - MULTILEVEL - INCL SPLIT LEV/FOYER':'Other','1-1/2 STORY - UNFINISHED ALL AGES':'Other'\
                                 ,'2-1/2 STORY ALL AGES':'Other'},
                                 'MSZoning':{'RH':'Other','C (all)':'Other'},
                                 'MiscFeature':{'Gar2':'Other','TenC':'Other'},
                                 'RoofMatl':{'Tar&Grv':'Other', 'WdShake':'Other', 'WdShng1':'Other', 'Metal':'Other', 'Roll':'Other', 'Membran':'Other'},
                                 'RoofStyle':{'Gambrel':'Other', 'Flat':'Other', 'Mansard':'Other', 'Shed':'Other'},
                                 'SaleCondition':{'Alloca':'Other','AdjLand':'Other'},
                                 'SaleType':{'ConLD':'Other', 'CWD':'Other', 'ConLI':'Other', 'ConLw':'Other', 'Oth':'Other', 'Con':'Other'}})


In [ ]:
train_test = pd.get_dummies(train_test)
# train_set = pd.get_dummies(train_set)
train_test = train_test.drop('Id',axis=1)
# train_set = train_set.drop('Id',axis=1)

In [ ]:
# correlation = train_test[:1453].corr(method='spearman').SalePrice.sort_values()
# correlation = pd.DataFrame(correlation)
# fig, axarr = matplotlib.pyplot.subplots(1,1,figsize=(70,240))
# barplot = sns.barplot(data=correlation, y=correlation.index, x='SalePrice', orient='h')
# barplot.tick_params(labelsize=60)


In [ ]:
# fig, axarr = plt.subplots(1,2,figsize=(17,7))
# sns.distplot(np.log(train_test.LotFrontage),ax=axarr[0])
# scipy.stats.probplot(np.log(train_test.LotFrontage), plot=plt)









In [ ]:
low_val = np.array([])
for clean in train_test.columns:
    if len(train_test[(train_test[clean].isnull()) | (train_test[clean]==0)]) >= len(train_test)*(1-0.001):
        low_val = np.append(low_val,clean)
        train_test = train_test.drop(clean, axis=1)


In [ ]:
for i in list(combinations(train_test.columns,2)):
    if i[0] in train_test.columns and i[1] in train_test.columns:
        corr = spearmanr(train_test[i[0]],train_test[i[1]])
        if np.abs(corr[0])>=0.9:
            if spearmanr(train_test.SalePrice[:1450],train_test[i[0]][:1450])[0]>\
            spearmanr(train_test.SalePrice[:1450],train_test[i[1]][:1450])[0]:
                train_test = train_test.drop(i[1],axis=1)
            else:
                train_test = train_test.drop(i[0],axis=1)
        else:
            continue
    
    else:
        continue


In [ ]:
# from sklearn.linear_model import LinearRegression
# import sys

# def variance_inflation_factor_test(X_array, X_exog):
#     X_exog1 = X_exog.copy()
#     X_array = X_array.drop(X_exog1.name, axis=1)
#     model = LinearRegression()
#     model.fit(X_array, X_exog)
#     return 1 / (1 - model.score(X_array, X_exog))

# for column in train_test.drop('SalePrice',axis=1).columns:
#     VIF_result = variance_inflation_factor_test(train_test.drop('SalePrice',axis=1), train_test[column])
#     if VIF_result > 10:
#         train_test = train_test.drop(column,axis=1)
# #         train_set = train_set.drop(column,axis=1)

In [ ]:

# # Create the RFE object and rank each pixel
# clf_rf_3 = RandomForestClassifier()      
# rfe = RFE(estimator=clf_rf_3, n_features_to_select=10, step=1)
# rfe = rfe.fit(train_X, train_y.astype(int))
# print('Chosen best 5 feature by rfe:',train_X.columns[rfe.support_])

In [ ]:
# find best scored 5 features

select_feature = SelectKBest(f_classif).fit(train_test.drop('SalePrice',axis=1)[:1450], np.log(train_test.SalePrice[:1450]).astype(int))

scores = pd.DataFrame({'Features':train_test.drop('SalePrice',axis=1)[:1450].columns,'Scores':select_feature.scores_}).sort_values(by='Scores',ascending=False)

scores = scores.set_index('Features')
scores['Position'] = range(1,len(scores)+1)
scores

In [ ]:
# a = train_test.drop(np.array(scores[scores.Scores>50000].Features),axis=1)
# a = a.drop('SalePrice',axis=1)
# b = train_test.drop(a,axis=1)
# b

In [ ]:
# # The "accuracy" scoring is proportional to the number of correct classifications
# AmountOfFeatures = np.array([])
# Features = np.array([])
# # for i in range(20):
# print(i)
# clf_rf_4 = RandomForestClassifier() 
# rfecv = RFECV(estimator=clf_rf_4, step=1, cv=5,scoring='accuracy')   #5-fold cross-validation
# rfecv = rfecv.fit(train_test.drop(['SalePrice'],axis=1)[:1453], np.log(train_test.SalePrice)[:1453].astype(int))
# AmountOfFeatures = np.append(AmountOfFeatures,rfecv.n_features_)
# Features = np.append(Features, train_test.drop(['SalePrice'],axis=1)[:1453].columns[rfecv.support_])
# #     print('Optimal number of features :', rfecv.n_features_)
# #     print('Best features :', train_test.drop(['SalePrice'],axis=1)[:1453].columns[rfecv.support_])

# print(pd.Series(Features).value_counts())
# print(AmountOfFeatures)


In [ ]:
# Features1= pd.Series(Features)
# Features1 = Features1.value_counts()
# Features1 = pd.DataFrame(Features1)
# Features1 = Features1.sort_index(ascending=True)
# Features1 = Features1.rename(columns={0:'Appearances'})
# final = pd.concat([Features1,scores],axis=1)
# final.sort_values(by='Scores',ascending=False)

In [ ]:
# # final.sort_values(by='Appearances',ascending=False)
# a = Features1[Features1.Appearances>=4]
# len(a)

In [ ]:
# a = train_test.drop(['SalePrice'],axis=1)[:1453].columns[rfecv.support_]
# b = train_test.drop(a,axis=1)
# b = b.drop('SalePrice',axis=1)
# train_test = train_test.drop(b,axis=1)


In [ ]:
# c = train_test.drop(a.index,axis=1)
# c = c.drop('SalePrice',axis=1)
# d = train_test.drop(c,axis=1)

In [ ]:
# from sklearn.linear_model import Lasso
# from xgboost import XGBRegressor


# x=[]
# y = []
# # for i in range(20):
# train_X, test_X, train_y, test_y = train_test_split(train_test.drop(['SalePrice'],axis=1)[:1453],\
#                                         np.log(train_test.SalePrice)[:1453],random_state=4)
# for feat in scores.index[:-1]:
#     train_X = train_X.drop(feat,axis=1)
#     test_X = test_X.drop(feat,axis=1)
#     model = Lasso(alpha=0.0001)
#     model.fit(train_X, train_y)
#     pred = np.exp(model.predict(test_X))
#     actual = np.exp(test_y)

#     actual = np.log(actual)
#     predicted = np.log(pred)
#     error1 = np.sqrt(np.sum(np.square(actual-predicted))/len(actual))
#     x.append(error1)

#     pred1 = np.exp(model.predict(train_X))
#     actual1 = np.exp(train_y)

#     actual1 = np.log(actual1)
#     predicted1 = np.log(pred1)
#     error2 = np.sqrt(np.sum(np.square(actual1-predicted1))/len(actual1))
#     y.append(error2)

# res = pd.DataFrame({'TrainSet':y,'TestSet':x})
# fig, axarr = plt.subplots(figsize=(20,8))
# sns.lineplot(data=res)
# # print('TRAIN DATA\n','Median:',np.median(y),'Max:',np.max(y),'Min:',np.min(y))
# # print('TEST DATA\n','Median:',np.median(x),'Max:',np.max(x),'Min:',np.min(x))
# # print('Difference between sets:',np.median(x)-np.median(y))
# # # a = pd.DataFrame({'alpha':[i],'error':[err]})
# # # error = pd.concat([error, a],axis=0)

# # # fig, axarr = plt.subplots(figsize=(15,7))
# # # error.plot.scatter(x = 'alpha', y = 'error',ax=axarr)

In [ ]:
# from sklearn.linear_model import Lasso
# from xgboost import XGBRegressor


# x=[]
# y = []
# # for i in range(20):
# train_X, test_X, train_y, test_y = train_test_split(train_test.drop(['SalePrice'],axis=1)[:1453],\
#                                     np.log(train_test.SalePrice)[:1453],random_state=1)

# model = XGBRegressor(learning_rate=0.01,
#                     n_estimators=1000,
#                     max_depth=3,
#                     subsample=0.8,
#                     colsample_bytree=0.3,
#                     gamma=0)

# model.fit(train_X, train_y)
# pred = np.exp(model.predict(test_X))
# actual = np.exp(test_y)

# actual = np.log(actual)
# predicted = np.log(pred)
# error1 = np.sqrt(np.sum(np.square(actual-predicted))/len(actual))
# x.append(error1)

# pred1 = np.exp(model.predict(train_X))
# actual1 = np.exp(train_y)

# actual1 = np.log(actual1)
# predicted1 = np.log(pred1)
# error2 = np.sqrt(np.sum(np.square(actual1-predicted1))/len(actual1))
# y.append(error2)

# res = pd.DataFrame({'TrainSet':y,'TestSet':x})

# # print('TRAIN DATA\n','Median:',np.median(y),'Max:',np.max(y),'Min:',np.min(y))
# # print('TEST DATA\n','Median:',np.median(x),'Max:',np.max(x),'Min:',np.min(x))
# # print('Difference between sets:',np.median(x)-np.median(y))
# # # a = pd.DataFrame({'alpha':[i],'error':[err]})
# # # error = pd.concat([error, a],axis=0)

# # # fig, axarr = plt.subplots(figsize=(15,7))
# # # error.plot.scatter(x = 'alpha', y = 'error',ax=axarr)

In [ ]:
# fig, axarr = plt.subplots(figsize=(15,10))
# sns.lineplot(data=res)

In [ ]:
from sklearn.linear_model import Lasso
from xgboost import XGBRegressor


x=[]
y = []
# for i in range(20):
train_X, test_X, train_y, test_y = train_test_split(train_test.drop(['SalePrice'],axis=1)[:1453],\
                                    np.log(train_test.SalePrice)[:1453],random_state=1)

error = pd.DataFrame({'alpha':[],'error':[]})
model = Lasso(alpha = 0.00010)
model.fit(train_X, train_y)
pred = np.exp(model.predict(test_X))
actual = np.exp(test_y)

actual = np.log(actual)
predicted = np.log(pred)
error1 = np.sqrt(np.sum(np.square(actual-predicted))/len(actual))
x.append(error1)

pred1 = np.exp(model.predict(train_X))
actual1 = np.exp(train_y)

actual1 = np.log(actual1)
predicted1 = np.log(pred1)
error2 = np.sqrt(np.sum(np.square(actual1-predicted1))/len(actual1))
y.append(error2)

print('TRAIN DATA\n','Median:',np.median(y),'Max:',np.max(y),'Min:',np.min(y))
print('TEST DATA\n','Median:',np.median(x),'Max:',np.max(x),'Min:',np.min(x))
print('Difference between sets:',np.median(x)-np.median(y))
# # a = pd.DataFrame({'alpha':[i],'error':[err]})
# # error = pd.concat([error, a],axis=0)

# # fig, axarr = plt.subplots(figsize=(15,7))
# # error.plot.scatter(x = 'alpha', y = 'error',ax=axarr)

In [ ]:
a = train_test.drop(scores.index[:120],axis=1)
a = a.drop('SalePrice',axis=1)
train_test = train_test.drop(a,axis=1)

In [ ]:
# from sklearn.linear_model import Lasso
# from xgboost import XGBRegressor

# # for i in range(20):
# #     train_X, test_X, train_y, test_y = train_test_split(train_test.drop(['SalePrice'],axis=1)[:1453],\
# #                                         np.log(train_test.SalePrice)[:1453],random_state=i)

# model = Lasso(alpha=0.0001,tol=0.03)
# model.fit(train_test[:1453].drop('SalePrice',axis=1), np.log(train_test.SalePrice[:1453]))
# pred = np.exp(model.predict(train_test[1453:].drop('SalePrice',axis=1)))
# # actual = np.exp(test_y)
# test_set = pd.read_csv(r'../input/test.csv')
# test_set = test_set.Id
# test_set = pd.concat([test_set,pd.Series(pred)],axis=1)
# test_set = test_set.rename(columns={0:'SalePrice'})
# test_set = test_set.set_index('Id')
# test_set.to_csv(r'submission.csv')

# # actual = np.log(actual)
# # predicted = np.log(pred)
# # print(np.sqrt(np.sum(np.square(actual-predicted))/len(actual)))

# # a = pd.DataFrame({'alpha':[i],'error':[err]})
# # error = pd.concat([error, a],axis=0)

# # fig, axarr = plt.subplots(figsize=(15,7))
# # error.plot.scatter(x = 'alpha', y = 'error',ax=axarr)



In [ ]:
# from sklearn.linear_model import Lasso
# from xgboost import XGBRegressor



# error = pd.DataFrame({'CutoffVal':[],'error':[]})

# for i in np.arange(0.001,0.01,0.0001):
#     train_pyk = train_test.copy()
#     for clean in train_pyk.columns:
#         if len(train_pyk[(train_pyk[clean].isnull()) | (train_pyk[clean]==0)]) >= len(train_pyk)*(1-i):
#             train_test = train_pyk.drop(clean, axis=1)

    
    
#     train_X, test_X, train_y, test_y = train_test_split(train_pyk.drop('SalePrice',axis=1)[:1453],\
#                                     np.log(train_pyk.SalePrice)[:1453],random_state=1)

    
    
#     model = Lasso(alpha = 0.00001, tol=0.1)
#     model.fit(train_X, train_y)
#     pred = np.exp(model.predict(test_X))
#     actual = np.exp(test_y)

#     actual = np.log(actual)
#     predicted = np.log(pred)
#     err = np.sqrt(np.sum(np.square(actual-predicted))/len(actual))
#     a = pd.DataFrame({'CutoffVal':[i],'error':[err]})
#     error = pd.concat([error, a],axis=0)

# fig, axarr = plt.subplots(figsize=(15,7))
# error.plot.scatter(x = 'CutoffVal', y = 'error',ax=axarr)

In [ ]:
# train_set = train_test.loc[:1453]
# correlation = train_set.corr(method='spearman').SalePrice.sort_values()
# correlation = pd.DataFrame(correlation)
# fig, axarr = matplotlib.pyplot.subplots(1,1,figsize=(70,320))
# barplot = sns.barplot(data=correlation, y=correlation.index, x='SalePrice', orient='h')
# barplot.tick_params(labelsize=60)


In [ ]:
# correlation

# spike_cols = [correlation.loc[col] for col in correlation.index if 'GarageYrBlt' in col]

# a = pd.DataFrame(spike_cols)
# # fig, axarr = matplotlib.pyplot.subplots(1,1,figsize=(40,120))
# # lol=sns.barplot(data=a, y='SalePrice', x=a.index, orient='h')
# # lol.tick_params(labelsize=60)
# a